In [17]:
# do this before anything else because you don't want to have to rerun this 
# every time you need to install something new - pain in the ass

# Install Earth Engine API
!pip install -q earthengine-api 

# import ee
import ee
# from ee.oauth import get_credentials_path
# print(get_credentials_path())

# Authenticate Earth Engine
ee.Authenticate(force=True)
print('GEE authenticated')

Enter verification code:  4/1ATX87lPineLb1a9RfHS4XXIZToPjbwGe2IvMse6WReNyNNzppYTnYIiXg2g



Successfully saved authorization token.
GEE authenticated


In [18]:
# Initialize Earth Engine
ee.Initialize(project='columbia-phd')
print('GEE initialized')

# import whatever else you need
from datetime import datetime
import os
import rasterio
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import ListedColormap
# GEEmap for visualization
!pip install -q geemap
# !pip install -q ipywidgets geemap
# !pip install -q ipywidgets geemap jupyterlab_widgets
import geemap
print('GEE imported')

GEE initialized
GEE imported


In [19]:
# define administrative aoi
# gaul = ee.FeatureCollection("FAO/GAUL/2015/level1") # get the FAO feature collection
# peru = gaul.filter(ee.Filter.eq('ADM0_NAME', 'Peru')) # Filter Peru first (ADM0_NAME = country)
# aoi = peru.filter(ee.Filter.eq('ADM1_NAME', 'Madre de Dios')) # Filter Madre de Dios region (ADM1_NAME = admin1 region)

square = ee.Feature(
    ee.Geometry.Polygon([
        [[-73.489604265885, -9.106738678620076],
          [-73.489604265885, -14.471148538041154],
          [-67.72178200026, -14.471148538041154],
          [-67.72178200026, -9.106738678620076]]
    ])
)
aoi = ee.FeatureCollection([square])

# set the tree cover extent parameters (we consider forest all that is at least 30% canopy cover, at least 0.5 ha continuous, this is consistent with Hansen data handling)
canopy_cover = ee.Number(30) # canopy cover threshold (%)
min_extent = ee.Number(6) # min size threshold (6 pixels ≈ 0.5 ha)

In [26]:
# load the Hansen Global Forest Change data layer
gfc2024 = ee.Image("UMD/hansen/global_forest_change_2024_v1_12")

# get tree cover extent in 2000
canopyCover_2000 = gfc2024.select("treecover2000")

# mask the tree cover according to our tree cover parameters defined above
canopyCover30 = canopyCover_2000.gte(canopy_cover).selfMask() # all pixels in 2000 with at least 30% canopy cover, as defined by Hansen et al
contArea = canopyCover30.connectedPixelCount()
extent2000 = contArea.gte(min_extent).selfMask() # all pixels in 2000 with at least 0.5 ha of continuous forest

# Get projection and scale for visualization later on
prj = gfc2024.projection()
scale = prj.nominalScale()
print(scale.getInfo())

27.829872698318393


In [27]:
# from the GFC dataset, get tree loss
treeLoss = gfc2024.select("loss")

# In the GFC dataset, any value in "loss" that is greater 0 idnicates a loss event. The value to this loss event indicates the year. We want no loss events, loss=0 
noLoss = treeLoss.eq(0)

# mask the 2000 extent by the noLoss
current_extent = extent2000.updateMask(noLoss)

# clip this to the aoi defined above
mdd_extent = current_extent.clip(aoi)

In [22]:
# get the nominal scale of the TerraClim data, our largest dataset
TerraClim = ee.ImageCollection('IDAHO_EPSCOR/TERRACLIMATE')
first = TerraClim.first()
TC_scale = first.projection().nominalScale()

print(TC_scale.getInfo())

4638.312116386398


In [23]:
# Create map
m = geemap.Map()
m.centerObject(aoi, zoom=7)

# add to map
m.addLayer(aoi, {}, "AOI")
m.addLayer(mdd_extent, 
           {}, 
           "Hansen")

m

Map(center=[-11.794669076434946, -70.60569313307249], controls=(WidgetControl(options=['position', 'transparen…

In [29]:
# rescale and reproject the image to the TC data scale
mdd_extent_mode = mdd_extent.reduceResolution(
    reducer=ee.Reducer.mode(),  # aggregation function
    maxPixels=65536
).reproject(
    crs=mdd_extent.projection(),
    scale=5000
)

# export the gee image to the python environment
out = "/home/jovyan/MLEAEEE4000-DroughtAmazon2/data/hansen_datamask_v2.tif"

geemap.ee_export_image(
    mdd_extent,
    filename=out,
    scale=scale, # assign the scale value of the TC data
    region=aoi.geometry()
)

# task = ee.batch.Export.image.toDrive(
#     image=mdd_extent,
#     description='mdd_extent_mode_5km',
#     folder='gee_exports',        # optional
#     fileNamePrefix='mdd_extent_mode_5km',
#     scale=5000,
#     region=aoi.geometry(),
#     maxPixels=1e13,               # important
#     # pyramidingPolicy = 'mode'
# )
# task.start()

print("Export started!")


Generating URL ...
An error occurred while downloading.
Total request size (993434176 bytes) must be less than or equal to 50331648 bytes.


EEException: Unknown configuration options: {'pyramidingPolicy': 'mode'}.

In [25]:
# Create map
m = geemap.Map()
m.centerObject(aoi, zoom=7)

# add to map
m.addLayer(aoi, {}, "AOI")
m.addLayer(mdd_extent_mode, 
           {}, 
           "Hansen")

m

Map(center=[-11.794669076434946, -70.60569313307249], controls=(WidgetControl(options=['position', 'transparen…